In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-29,190 Morgan,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-30,190 Morgan,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-31,190 Morgan,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-01,190 Morgan,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-02,190 Morgan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-27,SYS038,0
137084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-28,SYS038,0
137085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-29,SYS038,0
137086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-30,SYS038,0


In [1]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 3, 15, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

ModuleNotFoundError: No module named 'src'

In [12]:
import numpy as np

class BaselineModelPreviousHourCitiBike:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test["rides_t-1"]

In [13]:
model = BaselineModelPreviousHourCitiBike()
predictions = model.predict(X_test)

In [14]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.7117


In [15]:
from src.experiments_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
load_dotenv() 

mlflow = set_mlflow_tracking()

INFO:src.experiments_utils:MLflow tracking URI and credentials set.


In [16]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousHourCitiBike", "mean_absolute_error", score=test_mae)

2025/05/10 10:33:30 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousHourCitiBike' does not exist. Creating a new experiment.
INFO:src.experiments_utils:Experiment set to: BaselineModelPreviousHourCitiBike
INFO:src.experiments_utils:Logged mean_absolute_error: 0.7116944204152249
f:\DushyanthData\Applied_Machine_Learning\Final_Project\finalProject\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.or

2025/05/10 10:35:37 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORK_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'BaselineModelPreviousHourCitiBike'.
2025/05/10 10:38:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousHourCitiBike, version 1
Created version '1' of model 'BaselineModelPreviousHourCitiBike'.
INFO:src.experiments_utils:Model logged with name: BaselineModelPreviousHourCitiBike


🏃 View run incongruous-tern-387 at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/4/runs/28884c9b22dd4f1a981ccc92daca1d10
🧪 View experiment at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/4


In [17]:
import numpy as np

class BaselineModelPreviousWeekCitiBike:

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        return X_test[f"rides_t-{7*24}"]

In [18]:
model = BaselineModelPreviousWeekCitiBike()
predictions = model.predict(X_test)

In [19]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.5926


In [20]:
log_model_to_mlflow(model, X_test, "BaselineModelPreviousWeekCitiBike", "mean_absolute_error", score=test_mae)

2025/05/10 10:40:28 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelPreviousWeekCitiBike' does not exist. Creating a new experiment.
INFO:src.experiments_utils:Experiment set to: BaselineModelPreviousWeekCitiBike
INFO:src.experiments_utils:Logged mean_absolute_error: 0.5926146193771626
f:\DushyanthData\Applied_Machine_Learning\Final_Project\finalProject\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.or

Successfully registered model 'BaselineModelPreviousWeekCitiBike'.
2025/05/10 10:45:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelPreviousWeekCitiBike, version 1
Created version '1' of model 'BaselineModelPreviousWeekCitiBike'.
INFO:src.experiments_utils:Model logged with name: BaselineModelPreviousWeekCitiBike


🏃 View run unleashed-moose-194 at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/5/runs/3ea9f9a483994455a29c536a8f925aed
🧪 View experiment at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/5


In [21]:
import numpy as np
import pandas as pd

class BaselineModelLast4WeeksCitiBike:
    """
    A baseline model that predicts the average of the last 4 weeks (28 days)
    for each test instance.
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        """
        The fit method is not used in this baseline model as it does not learn
        from the training data.
        """
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """
        Predicts the average of the last 4 weeks (28 days) for each test instance.

        Parameters:
            X_test (pd.DataFrame): The test DataFrame containing lagged features
                                   (e.g., rides_t-{7*24}, rides_t-{14*24}, etc.).

        Returns:
            np.array: An array of predictions based on the average of the last 4 weeks.
        """
        # Define the columns for the last 4 weeks
        last_4_weeks_columns = [
            f"rides_t-{7*24}",  # 1 week ago
            f"rides_t-{14*24}", # 2 weeks ago
            f"rides_t-{21*24}", # 3 weeks ago
            f"rides_t-{28*24}"  # 4 weeks ago
        ]

        # Ensure the required columns exist in the test DataFrame
        for col in last_4_weeks_columns:
            if col not in X_test.columns:
                raise ValueError(f"Missing required column: {col}")

        # Calculate the average of the last 4 weeks
        predictions = X_test[last_4_weeks_columns].mean(axis=1)

        return predictions.to_numpy()

In [22]:
model = BaselineModelLast4WeeksCitiBike()
predictions = model.predict(X_test)

In [23]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.4983


In [24]:
log_model_to_mlflow(model, X_test, "BaselineModelLast4WeeksCitiBike", "mean_absolute_error", score=test_mae)

2025/05/10 10:46:34 INFO mlflow.tracking.fluent: Experiment with name 'BaselineModelLast4WeeksCitiBike' does not exist. Creating a new experiment.
INFO:src.experiments_utils:Experiment set to: BaselineModelLast4WeeksCitiBike
INFO:src.experiments_utils:Logged mean_absolute_error: 0.4982563797577855
f:\DushyanthData\Applied_Machine_Learning\Final_Project\finalProject\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/do

Successfully registered model 'BaselineModelLast4WeeksCitiBike'.
2025/05/10 10:50:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: BaselineModelLast4WeeksCitiBike, version 1
Created version '1' of model 'BaselineModelLast4WeeksCitiBike'.
INFO:src.experiments_utils:Model logged with name: BaselineModelLast4WeeksCitiBike


🏃 View run clean-whale-673 at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/6/runs/92ffbd45fddf4581bea1e0db76e88476
🧪 View experiment at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/6


In [25]:
X_test[X_test['pickup_location_id']=="6140.05"] 

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id
18003,2,1,1,0,1,1,6,11,32,19,...,45,55,53,34,27,17,8,6,2024-03-15,6140.05
18004,4,0,0,0,1,0,0,3,6,2,...,31,56,48,28,25,11,6,7,2024-03-16,6140.05
18005,1,0,0,1,0,1,2,2,4,8,...,38,32,25,18,15,9,4,2,2024-03-17,6140.05
18006,2,0,0,0,0,1,0,5,7,10,...,34,39,30,21,13,8,6,2,2024-03-18,6140.05
18007,1,1,1,0,0,0,2,14,31,14,...,29,37,36,34,16,14,3,2,2024-03-19,6140.05
18008,2,0,0,0,0,4,7,23,34,25,...,29,45,43,37,28,11,4,2,2024-03-20,6140.05
18009,1,1,0,0,0,3,8,13,23,21,...,23,30,40,26,24,10,4,3,2024-03-21,6140.05
18010,0,0,0,0,0,0,5,4,33,22,...,45,35,41,34,14,11,6,6,2024-03-22,6140.05
18011,1,1,3,1,0,1,2,2,6,13,...,32,41,35,24,15,11,5,5,2024-03-23,6140.05
18012,2,3,3,0,1,0,0,3,9,8,...,2,8,17,12,12,2,7,7,2024-03-24,6140.05


In [26]:
from src.plot_utils import plot_aggregated_time_series

plot_aggregated_time_series(X_test, y_test, 18003, predictions)